In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import pandas as pd

In [55]:
from langchain.globals import set_debug
set_debug(False)

In [18]:
loader = UnstructuredURLLoader(urls=["https://www.apple.com"])
first_document = loader.load()

In [19]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 0)
chunks = splitter.split_documents(first_document)

In [ ]:
prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
    
    {context}
    
    Question: {question}
    
    Answer the question and cite the relevant sources from the context. if you do not know the answer, Say so""")

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-4o-mini")
faiss_db = FAISS.from_documents(chunks, embeddings)

faiss_db.save_local("faiss_db")

In [56]:
def create_qa_chain(retriever, llm):
    chain = RunnableParallel( {
        "context": retriever,
        "question": RunnablePassthrough()
    }) | prompt | llm | StrOutputParser()

    return chain

In [21]:
df = pd.read_csv("sample_text.csv")

In [28]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")

In [29]:
vectors = encoder.encode(df["text"])

In [31]:
dim = vectors.shape[1]

In [34]:
import faiss, numpy

index = faiss.IndexFlatL2(dim)

In [35]:
index.add(vectors)

In [37]:
index.search(numpy.array(encoder.encode("i want to buy a polo t-shirt")).reshape(1,-1), k=2)

(array([[1.3844836, 1.4039094]], dtype=float32), array([[3, 2]], dtype=int64))

In [57]:
qa_chain = create_qa_chain(faiss_db.as_retriever(), llm)

In [58]:
qa_chain.invoke("is apple a fruit?")

'The context provided does not contain any information about apples as a fruit. It focuses on Apple products like the iPhone, AirPods, and MacBook. Therefore, based on the provided context, there is no evidence to support or refute the question about whether an apple is a fruit.'